In [3]:
import requests
import re
import time
import os
import pandas as pd
import json
import sys
import warnings
import numpy as np
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Hannanum
from datetime import date

In [2]:
def get_page_sources_youtube(url):
    
    browser = webdriver.Chrome("./chromedriver.exe")
    page_sources = []
    
    # Move to the page
    url = url
    browser.get(url)
    
    # Scroll down
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")
    
    # Iterate until end script of the website
    while True:
        
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(4.0)
        
        new_page_height = browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        if new_page_height == last_page_height:

            break
        
        else:
            print("Continue")

        last_page_height = new_page_height

        print("End of scrolling")
        
        page_source = browser.page_source
        page_sources.append(page_source)  
    
    return page_sources

In [3]:
def get_info(url):

    url = url
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")   
    browser = webdriver.Chrome("./chromedriver.exe")

    info = []
    
    video_name = soup.find('meta', itemprop="name")['content']
    num_views = soup.find('meta', itemprop="interactionCount")['content']
    date_published = soup.find('meta', itemprop="datePublished")['content']
    
    info.append([video_name, num_views, date_published])
    
    video_info = pd.DataFrame(index = ["Info"], columns = ['Video_name', 'Num_views', 'Date_published'], 
                              data = info)
    return video_info

def get_Comment_Like(url):
    
    browser = webdriver.Chrome("./chromedriver.exe")
    comment = []
    comment_date= []
    num_like = []

    # Move to the page
    url = url
    browser.get(url)
    time.sleep(5)
    # Scroll down and wait to get the reply
    
    browser.execute_script("window.scrollTo(0, 600);")
    time.sleep(10)
    browser.execute_script("window.scrollTo(600, 0);")
    time.sleep(5)
    browser.execute_script("window.scrollTo(10, document.documentElement.scrollHeight);")
    time.sleep(5)
    
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")
    
    # Iterate until end script of the website
    while True:
        
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(5.0)
        
        new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        
        if new_page_height == last_page_height:
            print("End scrolling")
            print("---"*30)
            print("Start Scrapping")
            break
        
        else:
            print("Iterating...")
            
        last_page_height = new_page_height
       
    # Get the comment
    for comments in browser.find_elements_by_id("content-text"):
        comment.append(comments.text)
        
    # Get year of the comment("~months,weeks,days" will be 2022 and others will be "2022 - ~years" in this code)
    for dates in browser.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a'):
        comment_date.append(dates.text)
        
    # Get the num of likes
    for likes in browser.find_elements_by_id("vote-count-middle"):

        if likes.text:
            num_like.append(likes.text)
        else:
            num_like.append("0")
            
            
            
    browser.quit()    
    # Make the dataframe with scrapped data
    video_content = pd.DataFrame({"Comment": comment, "Comment_date": comment_date, "Likes": num_like})
    video_content.Comment_date = video_content.Comment_date.apply(change_year)
    video_content.Likes = video_content.Likes.apply(convert_str_to_number)
    
    return video_content

def convert_str_to_number(x):
    
    total_stars = 0
    num_map = {'천': 1000, '만': 10000, 'K':1000, 'M':1000000, 'B':1000000000}
    
    if x.isdigit():
        total_stars = int(x)
    else:
        if len(x) > 1:
            total_stars = float(x[:-1]) * num_map.get(x[-1].upper(), 1)
            
    return int(total_stars)

def change_year(w):
    w = w.strip()
    w = re.sub(r"\(수정됨\)", " ", w)
    w = re.sub(r"전", " ", w)
    w = re.sub(r"1년", "2021", w) # Change 1 year ago to 2021
    w = re.sub(r"1 years", "2021", w) # Change 1 year ago to 2021
    w = re.sub(r"2년", "2020", w) # Change 2 year ago to 2020
    w = re.sub(r"2 years", "2020", w) # Change 2 year ago to 2020
    w = re.sub(r"3년", "2019", w) # Change 3 year ago to 2019
    w = re.sub(r"3 years", "2019", w) # Change 3 year ago to 2019
    w = w.strip()
    w = re.sub("([0-9]+)([^0-9]+$)", "2022", w) # Change any other period to 2022
    return w

### Scrpae the VR concert comments

In [4]:
url = "https://www.youtube.com/watch?v=wYeFAlVC8qU"
travis_info = get_info(url)
travis_df = get_Comment_Like(url)

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...

In [8]:
#tavis_df = pd.read_csv("comment_data.csv")
travis_df["class"] = "vr_concert"
travis_df["singer"] = "travis_scott"

In [9]:
travis_df

,Comment,Comment_date,Likes,class,singer
0,"Alright, I don’t play fortnite anymore but we ...",2021,1400,vr_concert,travis_scott
1,This was the best event of all time ! \nI have...,2022,87,vr_concert,travis_scott
2,The devs are insane that they got this great v...,2022,57,vr_concert,travis_scott
3,"Amazing, I miss this time, I loved this season...",2022,3,vr_concert,travis_scott
4,Me acuerdo que este fue el primer evento que v...,2022,60,vr_concert,travis_scott
...,...,...,...,...,...
5135,Vengo del concierto de Ariana grande y sin du...,2021,72,vr_concert,travis_scott
5136,Imagine being in this game and not knowing tha...,2020,70,vr_concert,travis_scott
5137,But imagine if he did Can’t Say with Don Toliv...,2020,33,vr_concert,travis_scott
5138,I'm gonna tell my children that i was in this ...,2020,330,vr_concert,travis_scott


In [10]:
# comments of Ariana Grande
url = "https://www.youtube.com/watch?v=RiM0moNk74o&ab_channel=ArianaGrande"
ariana_info = get_info(url)
ariana_df = get_Comment_Like(url)

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrap

In [11]:
ariana_df["class"] = "vr_concert"
ariana_df["singer"] = "ariana_grande"
ariana_df

,Comment,Comment_date,Likes,class,singer
0,EPICO! <3 🇲🇽,2021,807,vr_concert,ariana_grande
1,This was one of the most beautiful moments in ...,2022,15,vr_concert,ariana_grande
2,This was one of the better concerts from Fortn...,2022,56,vr_concert,ariana_grande
3,The concert was a few months ago yet watching ...,2022,668,vr_concert,ariana_grande
4,"The transition from “The Way,” to “Positions,”...",2021,1600,vr_concert,ariana_grande
...,...,...,...,...,...
1382,travis scott event was better,2022,2,vr_concert,ariana_grande
1383,Meh,2022,1,vr_concert,ariana_grande
1384,I literally hate this,2022,1,vr_concert,ariana_grande
1385,Travis is better,2022,2,vr_concert,ariana_grande


In [12]:
url = "https://www.youtube.com/watch?v=iTiBp-ORNEo&ab_channel=Faiz"
ariana_info_2 = get_info(url)
ariana_df_2 = get_Comment_Like(url)
ariana_df_2["singer"] = "ariana_grande"
ariana_df_2["class"] = "vr_concert"

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping


In [13]:
ariana_df_2

,Comment,Comment_date,Likes,singer,class
0,What an event! What are your thoughts guys? I ...,2021,1400,ariana_grande,vr_concert
1,I believe she incorporated many artists who ha...,2021,194,ariana_grande,vr_concert
2,I always love the concert events. Each one up ...,2021,364,ariana_grande,vr_concert
3,It's the first time I see a Fortnite event. I ...,2021,53,ariana_grande,vr_concert
4,Seeing these live events only make me question...,2021,408,ariana_grande,vr_concert
...,...,...,...,...,...
1182,That's just piece of crap compared to Travis S...,2022,0,ariana_grande,vr_concert
1183,Booooooooooooooo,2021,0,ariana_grande,vr_concert
1184,Ñ,2022,0,ariana_grande,vr_concert
1185,And it sucks,2022,1,ariana_grande,vr_concert


In [14]:
ariana_comment = pd.concat([ariana_df, ariana_df_2]).reset_index(drop = True)
ariana_comment

,Comment,Comment_date,Likes,class,singer
0,EPICO! <3 🇲🇽,2021,807,vr_concert,ariana_grande
1,This was one of the most beautiful moments in ...,2022,15,vr_concert,ariana_grande
2,This was one of the better concerts from Fortn...,2022,56,vr_concert,ariana_grande
3,The concert was a few months ago yet watching ...,2022,668,vr_concert,ariana_grande
4,"The transition from “The Way,” to “Positions,”...",2021,1600,vr_concert,ariana_grande
...,...,...,...,...,...
2569,That's just piece of crap compared to Travis S...,2022,0,vr_concert,ariana_grande
2570,Booooooooooooooo,2021,0,vr_concert,ariana_grande
2571,Ñ,2022,0,vr_concert,ariana_grande
2572,And it sucks,2022,1,vr_concert,ariana_grande


In [16]:
# comments of trimix
url = "https://www.youtube.com/watch?v=pSGFzZJMNcc&ab_channel=Trimix"
trimix_info = get_info(url)
trimix_df = get_Comment_Like(url)
trimix_df["singer"] = "trimix"
trimix_df["class"] = "vr_concert"

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping


In [17]:
trimix_df

,Comment,Comment_date,Likes,singer,class
0,Subscribe if you enjoyed! 999 Forever.,2022,4700,trimix,vr_concert
1,0:00 lean wit me \n1:03 robbery \n1:49 Legends...,2022,135,trimix,vr_concert
2,"As a fan of Juice since 2017, this is fvckin b...",2022,289,trimix,vr_concert
3,i got chills seeing someone put effort into ma...,2022,2,trimix,vr_concert
4,Pure Art,2022,2,trimix,vr_concert
...,...,...,...,...,...
1322,Can you make a FGTeeV,2022,0,trimix,vr_concert
1323,Him,2022,0,trimix,vr_concert
1324,Noah Play make a funny kill dummy,2022,0,trimix,vr_concert
1325,999 666,2022,0,trimix,vr_concert


In [18]:
# comments of Marshmello
url = "https://www.youtube.com/watch?v=NBsCzN-jfvA&list=PLd_An9ufLQrp1Fc0Sf4NUjdxRoh3U4aIi&ab_channel=Marshmello"
marshmello_info = get_info(url)
marshmello_df = get_Comment_Like(url)
marshmello_df["singer"] = "marshmello"
marshmello_df["class"] = "vr_concert"

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping


In [19]:
marshmello_df

,Comment,Comment_date,Likes,singer,class
0,Be Kind w/ Halsey - OUT NOW https://youtu.be/...,2019,70000,marshmello,vr_concert
1,2 years ago? I still remember waking up so hyp...,2021,2200,marshmello,vr_concert
2,I miss the hype surrounding this so much...wha...,2022,30,marshmello,vr_concert
3,Sinto falta disso passa uma vibe tão boa de an...,2022,9,marshmello,vr_concert
4,This event was crazy I was there to it was so...,2022,9,marshmello,vr_concert
...,...,...,...,...,...
1374,,2022,1,marshmello,vr_concert
1375,Marshmallow is better,2022,0,marshmello,vr_concert
1376,"can i play fortnite with you i,am a big fan",2022,0,marshmello,vr_concert
1377,хуйня канцерт с трєвісом скоттом пізже було на...,2022,0,marshmello,vr_concert


In [63]:
marshmello_info = marshmello_info.T
marshmello_info["num_comments"] = len(marshmello_df)
marshmello_info = marshmello_info.T 
marshmello_info

,Info
Video_name,Marshmello Holds First Ever Fortnite Concert L...
Num_views,62605389
Date_published,2019-02-02
num_comments,1379


In [20]:
#df = pd.read_csv('Comment_data.csv')
#df["class"] = "vr_concert"
#df["singer"] = "travis_scott"
df = pd.concat([travis_df, ariana_comment,trimix_df,marshmello_df]).reset_index(drop = True)
df = df[df["Comment"].notnull()]
df = df.reset_index(drop  = True)
vr_comment = df.copy()
vr_comment.to_csv("vr_comment.csv", index = False, encoding = "utf-8-sig")

In [21]:
# Concat data info
#df_info = pd.read_csv("data_info.csv")
df_info = pd.concat([travis_info.T, marshmello_info.T, ariana_info.T, ariana_info_2.T, trimix_info.T]).reset_index(drop=True)
df_info["scrapped_date"] = date.today()
df_info.to_csv("data_info.csv", encoding = "utf-8-sig", index = True)

In [45]:
tavis_af_gn_info.T

,Video_name,Num_views,Date_published
Info,TRAVIS SCOTT - ASTROWORLD FESTIVAL 2021 - UTOP...,4297567,2021-11-06


### Scrpae the general concert comments

In [62]:
# Travis Scott
urls = ["https://www.youtube.com/watch?v=1Q1AF00SqXE&t=540s",
       "https://www.youtube.com/watch?v=9iIFJkPX5M4&t=13s",
       "https://www.youtube.com/watch?v=kjCxY5kQOr0",
       "https://www.youtube.com/watch?v=EwnwSNR4_HU&t=199s",
       "https://www.youtube.com/watch?v=HDd9ClPUXQQ",
       "https://www.youtube.com/watch?v=oyK1W1ux0pA",
       "https://www.youtube.com/watch?v=aksMDWoFxJ8",
       "https://www.youtube.com/watch?v=K0pZdahSxx4", 
       "https://www.youtube.com/watch?v=PdaMknV6oKE&t=58s",
       ]

tavis_af_gn_info = get_info(urls[0])
tavis_af_gn_df = get_Comment_Like(urls[0])
tavis_af_gn_info["num_comments"] = len(tavis_af_gn_df)

for i in range(1, len(urls)):
    new_info = get_info(urls[i])
    new_df = get_Comment_Like(urls[i])
    new_info["num_comments"] = len(new_df)
    tavis_af_gn_info = pd.concat([tavis_af_gn_info, new_info])
    tavis_af_gn_df = pd.concat([tavis_af_gn_df, new_df])

tavis_af_gn_info["singer"] = "travis_scott"
tavis_af_gn_df["class"] = "general_concert_after_covid"
tavis_af_gn_df.reset_index(drop = True, inplace = True)

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
End scrolling
--------------------

In [82]:
tavis_af_gn_info["scrapped_date"] = date.today()
tavis_af_gn_info.to_csv("travis_af_info.csv")
tavis_af_gn_df.to_csv("travis_af_comment.csv")

In [76]:
urls = ["https://www.youtube.com/watch?v=-mC3KQse1eY&t=71s",
       "https://www.youtube.com/watch?v=k_4vtPLxx90",
       "https://www.youtube.com/watch?v=4aCy85KCk_Q",
       "https://www.youtube.com/watch?v=ReDxOqEOHks",
       "https://www.youtube.com/watch?v=r4tBdCkoyFk",
       "https://www.youtube.com/watch?v=hghXUF7zGXw",
       "https://www.youtube.com/watch?v=PdaMknV6oKE&t=58s",
       "https://www.youtube.com/watch?v=p6GmNEtDiAI&t=4s",
       "https://www.youtube.com/watch?v=_o3awcJC7Hg",
       "https://www.youtube.com/watch?v=iaFKqyP_V88",
       "https://www.youtube.com/watch?v=k_4vtPLxx90&t=31s"]

tavis_pre_gn_info = get_info(urls[0])
tavis_pre_gn_df = get_Comment_Like(urls[0])
tavis_pre_gn_info["num_comments"] = len(tavis_pre_gn_df)

for i in range(1, len(urls)):
    new_info = get_info(urls[i])
    new_df = get_Comment_Like(urls[i])
    new_info["num_comments"] = len(new_df)
    tavis_pre_gn_info = pd.concat([tavis_pre_gn_info, new_info])
    tavis_pre_gn_df = pd.concat([tavis_pre_gn_df, new_df])

tavis_pre_gn_info["singer"] = "travis_scott"
tavis_pre_gn_df["class"] = "general_concert_pre_covid"
tavis_pre_gn_df.reset_index(drop = True, inplace = True)
tavis_pre_gn_df

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iteratin

,Comment,Comment_date,Likes,class
0,**SET-LIST**\n0:10-0:39 • ASTROWORLD introduct...,2022,20000,general_concert_pre_covid
1,As a 48 year old white lady who grew up on met...,2019,3400,general_concert_pre_covid
2,"THIS is what Travis concerts are about, great ...",2021,1200,general_concert_pre_covid
3,literally rewatching every astro concert rn. ...,2021,1100,general_concert_pre_covid
4,"still his best set, crowd was on another level",2021,833,general_concert_pre_covid
...,...,...,...,...
4488,yeas,2020,1,general_concert_pre_covid
4489,Shiiiiiiiiii,2020,0,general_concert_pre_covid
4490,Yooo,2020,1,general_concert_pre_covid
4491,2000th like,2020,0,general_concert_pre_covid


In [83]:
tavis_pre_gn_info["scrapped_date"] = date.today()
tavis_pre_gn_info.to_csv("travis_pre_info.csv")
tavis_pre_gn_df.to_csv("travis_pre_comment.csv")

In [4]:
# Arian Grande
urls = ["https://www.youtube.com/watch?v=nxN-gCtpUmA",
       "https://www.youtube.com/watch?v=s37x2VSZrLw",
       "https://www.youtube.com/watch?v=tXmKt-90jYw",
       "https://www.youtube.com/watch?v=DWsRX8G42N4",
       "https://www.youtube.com/watch?v=nxN-gCtpUmA&t=3s",
       "https://www.youtube.com/watch?v=VronettmNRI&t=1146s",
       "https://www.youtube.com/watch?v=fLFvbwrWLQY",
       "https://www.youtube.com/watch?v=r2XJ9P1NvJc",
       "https://www.youtube.com/watch?v=ZTF6vF7nHlM",
       "https://www.youtube.com/watch?v=dDIJBIZtTyM",
       "https://www.youtube.com/watch?v=F6OOVNethn8",
       "https://www.youtube.com/watch?v=uL52DajINdY"]

ariana_af_gn_info = get_info(urls[0])
ariana_af_gn_df = get_Comment_Like(urls[0])
ariana_af_gn_info["num_comments"] = len(ariana_af_gn_df)

for i in range(1, len(urls)):
    new_info = get_info(urls[i])
    new_df = get_Comment_Like(urls[i])
    new_info["num_comments"] = len(new_df)
    ariana_af_gn_info = pd.concat([ariana_af_gn_info, new_info])
    ariana_af_gn_df = pd.concat([ariana_af_gn_df, new_df])

ariana_af_gn_info["singer"] = "ariana_grande"
ariana_af_gn_df["class"] = "general_concert_after_covid"
ariana_af_gn_df.reset_index(drop = True, inplace = True)
ariana_af_gn_df

Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
--------------------

,Comment,Comment_date,Likes,class
0,"Обожаю тебя, Адриана!!! Ты такая единственная ...",2022,0,general_concert_after_covid
1,so fantastic you get them girl!!!,2021,9,general_concert_after_covid
2,Eu amo a nossa ariana grande,2021,3,general_concert_after_covid
3,OMG ITS ARIANA GRANDE!!!!!!!!!!!!!!!!!!!!!! I ...,2021,6,general_concert_after_covid
4,It's not 2021,2021,6,general_concert_after_covid
...,...,...,...,...
10582,Shite,2022,0,general_concert_after_covid
10583,I came here because her fans keep telling me I...,2022,0,general_concert_after_covid
10584,I really never thought this girl would go this...,2022,0,general_concert_after_covid
10585,Você não è a Ariana grande vc é cópia vc tá im...,2022,0,general_concert_after_covid


In [5]:
urls = ["https://www.youtube.com/watch?v=A85aEaZuxeY", 
       "https://www.youtube.com/watch?v=LHtgIWSUPew",
       "https://www.youtube.com/watch?v=sKm4YqadSGw",
       "https://www.youtube.com/watch?v=cshg5ktpPtI",
       "https://www.youtube.com/watch?v=D3q1ensbHHo&t=289s",
       "https://www.youtube.com/watch?v=e7wYDk8QbGA",
       "https://www.youtube.com/watch?v=vRf3azp1pak",
       ]

ariana_pre_gn_info = get_info(urls[0])
ariana_pre_gn_df = get_Comment_Like(urls[0])
ariana_pre_gn_info["num_comments"] = len(ariana_af_gn_df)

for i in range(1, len(urls)):
    new_info = get_info(urls[i])
    new_df = get_Comment_Like(urls[i])
    new_info["num_comments"] = len(new_df)
    ariana_pre_gn_info = pd.concat([ariana_pre_gn_info, new_info])
    ariana_pre_gn_df = pd.concat([ariana_pre_gn_df, new_df])

ariana_pre_gn_info["singer"] = "ariana_grande"
ariana_pre_gn_df["class"] = "general_concert_pre_covid"
ariana_pre_gn_df.reset_index(drop = True, inplace = True)
ariana_pre_gn_df

Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
--------------------

,Comment,Comment_date,Likes,class
0,"We we're so lucky we got to attend, it was a w...",2022,761,general_concert_pre_covid
1,Always wear black and pink on the 22nd of May ...,2021,77,general_concert_pre_covid
2,Ariana grande is a queen,2022,609,general_concert_pre_covid
3,I’m \nin tears,2020,280,general_concert_pre_covid
4,My girlfriend was involved in this but came ou...,2020,558,general_concert_pre_covid
...,...,...,...,...
8861,Comibaja gora arroz feijão,2022,0,general_concert_pre_covid
8862,Oijte novela agora noite,2022,1,general_concert_pre_covid
8863,Allahu Akbar,2021,0,general_concert_pre_covid
8864,Madison beer Kat \nDante Justin Bieber,2021,2,general_concert_pre_covid


In [6]:
# Marshmello
urls = ["https://www.youtube.com/watch?v=8eXFvzMEldk&t=102s",
       "https://www.youtube.com/watch?v=21rwF5XKVpA",
       "https://www.youtube.com/watch?v=dhTrdfMDI9c",
       "https://www.youtube.com/watch?v=PaTcKvWQv5Y"]

marsh_af_gn_info = get_info(urls[0])
marsh_af_gn_df = get_Comment_Like(urls[0])
marsh_af_gn_info["num_comments"] = len(marsh_af_gn_df)

for i in range(1, len(urls)):
    new_info = get_info(urls[i])
    new_df = get_Comment_Like(urls[i])
    new_info["num_comments"] = len(new_df)    
    marsh_af_gn_info = pd.concat([marsh_af_gn_info, new_info])
    marsh_af_gn_df = pd.concat([marsh_af_gn_df,new_df])

marsh_af_gn_info["singer"] = "marshmello"
marsh_af_gn_df["class"] = "general_concert_after_covid"
marsh_af_gn_df.reset_index(drop = True, inplace = True)

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
---

In [61]:
marsh_af_gn_df

,Comment,Comment_date,Likes,class
0,Tracklist: https://1001.tl/264u9s41\nStream/Do...,2021,1600.0,general_concert_after_covid
1,I love you all so much. Hearing your beautiful...,2021,3200.0,general_concert_after_covid
2,never seen marshmello live but i do know this ...,2022,3.0,general_concert_after_covid
3,"The show, the mixing, the delivery, the energy...",2021,232.0,general_concert_after_covid
4,would love to go to a concert but even the fla...,2022,1.0,general_concert_after_covid
...,...,...,...,...
1961,Fortineterali,2022,0.0,general_concert_after_covid
1962,fortnite,2022,0.0,general_concert_after_covid
1963,meh live performance,2022,0.0,general_concert_after_covid
1964,Fala português tu não é rico aprende a falar p...,2022,0.0,general_concert_after_covid


In [7]:
urls = ["https://www.youtube.com/watch?v=cOerPk-jTZU",
       "https://www.youtube.com/watch?v=k9dRk-PcAYY",
       "https://www.youtube.com/watch?v=5HAOjNMzUs4",
       "https://www.youtube.com/watch?v=R9q5lBy2w1E&t=72s",
       "https://www.youtube.com/watch?v=L1-g_l7iRyY&t=1s",
       "https://www.youtube.com/watch?v=LrtmcxmQ3UM",
       "https://www.youtube.com/watch?v=JHuDWjTm_5k",
       ]

marsh_pre_gn_info = get_info(urls[0])
marsh_pre_gn_df = get_Comment_Like(urls[0])
marsh_pre_gn_info["num_comments"] = len(marsh_pre_gn_df)

for i in range(1, len(urls)):
    new_info = get_info(urls[i])
    new_df = get_Comment_Like(urls[i])
    new_info["num_comments"] = len(new_df) 
    marsh_pre_gn_info = pd.concat([marsh_pre_gn_info,new_info])
    marsh_pre_gn_df = pd.concat([marsh_pre_gn_df, new_df])

marsh_pre_gn_info["singer"] = "marshmello"
marsh_pre_gn_df["class"] = "general_concert_pre_covid"
marsh_pre_gn_df.reset_index(drop = True, inplace = True)

Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Itera

In [8]:
# jucie wrld
urls = ["https://www.youtube.com/watch?v=_VtRjiAxEdU",
       ]

jucie_af_gn_info = get_info(urls[0])
jucie_af_gn_df = get_Comment_Like(urls[0])
jucie_af_gn_info["num_comments"] = len(jucie_af_gn_df)

jucie_af_gn_info["singer"] = "trimix"
jucie_af_gn_df["class"] = "general_concert_after_covid"

Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping


In [9]:
urls = ["https://www.youtube.com/watch?v=zLnowyZfyCs",
       "https://www.youtube.com/watch?v=T8R8uD_kG8c",
       "https://www.youtube.com/watch?v=uZD1rGb9XbM",
       "https://www.youtube.com/watch?v=LXelyorhE-w",
       "https://www.youtube.com/watch?v=T8R8uD_kG8c",
       "https://www.youtube.com/watch?v=qpY0K2QJQyo",
       "https://www.youtube.com/watch?v=AcvZGyHZnRg"]

jucie_pre_gn_info = get_info(urls[0])
jucie_pre_gn_df = get_Comment_Like(urls[0])
jucie_pre_gn_info["num_comments"] = len(jucie_pre_gn_df)

for i in range(1, len(urls)):
    new_info = get_info(urls[i])
    new_df = get_Comment_Like(urls[i])
    new_info["num_comments"] = len(new_df) 
    jucie_pre_gn_info = pd.concat([jucie_pre_gn_info, new_info])
    jucie_pre_gn_df = pd.concat([jucie_pre_gn_df, new_df])

jucie_pre_gn_info["singer"] = "trimix"
jucie_pre_gn_df["class"] = "general_concert_pre_covid"
jucie_pre_gn_df.reset_index(drop = True, inplace = True)

Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping
Iterating...
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping


In [10]:
#tavis_af_gn_info.to_csv("travis_af_info.csv")
#tavis_af_gn_df.to_csv("travis_af_comment.csv")
tavis_af_gn_info = pd.read_csv("travis_af_info.csv")
tavis_af_gn_df = pd.read_csv("travis_af_comment.csv")

#tavis_pre_gn_info.to_csv("travis_pre_info.csv")
#tavis_pre_gn_df.to_csv("travis_pre_comment.csv")

tavis_pre_gn_info = pd.read_csv("travis_pre_info.csv")
tavis_pre_gn_df = pd.read_csv("travis_pre_comment.csv")


In [48]:
jucie_pre_gn_info["singer"] = "trimix"
jucie_af_gn_info["singer"] = "trimix"
marsh_pre_gn_info["singer"] = "marshmello"
marsh_af_gn_info["singer"] = "marshmello"
ariana_pre_gn_info["singer"] = "ariana_grande"
ariana_af_gn_info["singer"] = "ariana_grande"
tavis_pre_gn_info["singer"] = "travis_scott"
tavis_af_gn_info["singer"] = "travis_scott"

In [50]:
jucie_pre_gn_df["singer"] = "trimix"
jucie_af_gn_df["singer"] = "trimix"
marsh_pre_gn_df["singer"] = "marshmello"
marsh_af_gn_df["singer"] = "marshmello"
ariana_pre_gn_df["singer"] = "ariana_grande"
ariana_af_gn_df["singer"] = "ariana_grande"
tavis_pre_gn_df["singer"] = "travis_scott"
tavis_af_gn_df["singer"] = "travis_scott"

In [59]:
comment_info_pre_covid = comment_info_pre_covid[comment_info_pre_covid["num_comments"] != 0]
comment_info_after_covid = comment_info_after_covid[comment_info_after_covid["num_comments"] != 0]

In [61]:
comment_info_after_covid

,Video_name,Num_views,Date_published,num_comments,singer,scrapped_date
0,TRAVIS SCOTT - ASTROWORLD FESTIVAL 2021 - UTOP...,4297890,2021-11-06,1318,travis_scott,2022-12-22
1,Travis Scott Live At @Rolling Loud 2021 NYC,65744,2022-05-24,47,travis_scott,2022-12-22
2,Future brings out Travis Scott to preform at R...,170368,2022-07-23,125,travis_scott,2022-12-22
3,TRAVIS SCOTT - ROLLING LOUD FULL SET MIAMI 2021,324649,2021-07-25,174,travis_scott,2022-12-22
4,"Travis Scott x Drake - Knife Talk, TSU, N2Deep...",374159,2021-11-06,287,travis_scott,2022-12-22
5,Travis Scott & Drake (SICKO MODE) [FULL HD] AS...,81992,2021-11-05,26,travis_scott,2022-12-22
6,Travis Scott - MAFIA (Live Performance Astrowo...,179824,2021-11-16,138,travis_scott,2022-12-22
7,Travis Scott Rolling Loud NYC Live 2019 (full ...,5998,2019-10-13,4,travis_scott,2022-12-22
8,Travis Scott - Live at GOV BALL 2018 (Full Set),3514403,2020-06-25,954,travis_scott,2022-12-22
9,Ariana Grande Live Full Concert 2021,20223,2021-01-03,10,ariana_grande,2022-12-22


In [53]:
comment_data_pre_covid = pd.concat([tavis_pre_gn_df, ariana_pre_gn_df, marsh_pre_gn_df, jucie_pre_gn_df])
comment_data_pre_covid= comment_data_pre_covid[comment_data_pre_covid["Comment"].notnull()]
comment_data_pre_covid.reset_index(drop = True, inplace = True)
comment_info_pre_covid = pd.concat([tavis_pre_gn_info, ariana_pre_gn_info, marsh_pre_gn_info, jucie_pre_gn_info]).reset_index(drop=True)
comment_info_pre_covid["scrapped_date"] = date.today()

comment_data_after_covid = pd.concat([tavis_af_gn_df, ariana_af_gn_df, marsh_af_gn_df, jucie_af_gn_df])
comment_data_after_covid = comment_data_after_covid[comment_data_after_covid["Comment"].notnull()]
comment_data_after_covid.reset_index(drop = True, inplace = True)
comment_info_after_covid = pd.concat([tavis_af_gn_info, ariana_af_gn_info, marsh_af_gn_info, jucie_af_gn_info]).reset_index(drop=True)
comment_info_after_covid["scrapped_date"] = date.today()

In [62]:
comment_data_pre_covid.to_csv("comment_data_pre_covid.csv", index = False, encoding = "utf-8-sig")
comment_info_pre_covid.to_csv("video_info_pre_covid.csv", index = False, encoding = "utf-8-sig")
comment_data_after_covid.to_csv("comment_data_after_covid.csv", index = False, encoding = "utf-8-sig")
comment_info_after_covid.to_csv("video_info_after_covid.csv", index = False, encoding = "utf-8-sig")

In [66]:
vr_comment = pd.read_csv("vr_comment_fin.csv")
df_info = pd.read_csv("data_info_fin.csv")
comment_data_pre_covid

,Comment,Comment_date,Likes,class,singer
0,**SET-LIST**\n0:10-0:39 • ASTROWORLD introduct...,2022,20000.0,general_concert_pre_covid,travis_scott
1,As a 48 year old white lady who grew up on met...,2019,3400.0,general_concert_pre_covid,travis_scott
2,"THIS is what Travis concerts are about, great ...",2021,1200.0,general_concert_pre_covid,travis_scott
3,literally rewatching every astro concert rn. ...,2021,1100.0,general_concert_pre_covid,travis_scott
4,"still his best set, crowd was on another level",2021,833.0,general_concert_pre_covid,travis_scott
...,...,...,...,...,...
17045,999,2021,6.0,general_concert_pre_covid,trimix
17046,???,2021,0.0,general_concert_pre_covid,trimix
17047,999,2021,2.0,general_concert_pre_covid,trimix
17048,This ain’t what juice says in the real song bu...,2021,1.0,general_concert_pre_covid,trimix


In [72]:
df_info['type'] = "vr_concert"
comment_info_pre_covid['type'] = "general_concert_pre_covid"
comment_info_after_covid['type'] = "general_concert_after_covid"

In [73]:
whole_comment = pd.concat([vr_comment, comment_data_pre_covid, comment_data_after_covid]).reset_index(drop = True)
whole_info = pd.concat([df_info, comment_info_pre_covid, comment_info_after_covid]).reset_index(drop = True)

In [78]:
whole_comment["Likes"] = whole_comment["Likes"].astype(int)

In [79]:
whole_comment

,Comment,Comment_date,Likes,class,singer
0,"Alright, I don’t play fortnite anymore but we ...",2021,1400,vr_concert,travis_scott
1,This was the best event of all time ! \nI have...,2022,87,vr_concert,travis_scott
2,The devs are insane that they got this great v...,2022,57,vr_concert,travis_scott
3,"Amazing, I miss this time, I loved this season...",2022,3,vr_concert,travis_scott
4,Me acuerdo que este fue el primer evento que v...,2022,60,vr_concert,travis_scott
...,...,...,...,...,...
42680,He is alive tho,2021,1,general_concert_after_covid,trimix
42681,Is that the real im a fan,2021,0,general_concert_after_covid,trimix
42682,Ty lol,2021,1,general_concert_after_covid,trimix
42683,This a old concert,2021,1,general_concert_after_covid,trimix


In [80]:
whole_comment.to_csv("whole_comment_data.csv", index = False, encoding = "utf-8-sig")
whole_info.to_csv("whole_video_info.csv", index = False, encoding = "utf-8-sig")

## Find the language and translate the comment to English

In [4]:
df = pd.read_csv("whole_comment_data_en.csv", encoding = "utf-8-sig")
#df = df[df["Comment"]!=" "]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41876 entries, 0 to 41875
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Comment       41876 non-null  object
 1   Comment_date  41876 non-null  int64 
 2   Likes         41876 non-null  int64 
 3   class         41876 non-null  object
 4   singer        41876 non-null  object
 5   Comment_en    41876 non-null  object
 6   en            41876 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 2.2+ MB


In [5]:
from googletrans import Translator

translator = Translator()
#vr_comment = pd.read_csv("vr_comment_data.csv") ## Have to change
#df = vr_comment.copy()

#df["Comment_en"] = ""
#df["en"] = 1

comment_en = []
en = []
error = []

try:
    for i in range(0, len(df)):
        if translator.translate(df.iloc[i][0], dest="en").src == 'en':
            en.append(1)
            comment_en.append(df.iloc[i][0])
            time.sleep(1.5)
        else:
            eng_text = translator.translate(df.iloc[i][0], dest="en").text
            en.append(0)
            comment_en.append(eng_text)
            time.sleep(1.5)
except:    
    comment_en.append(df.iloc[i][0])
    en.append(0)
    error.append(i)
    print(i)
    pass

0


In [5]:
df["Comment_en"] = comment_en
df["en"] = en

In [9]:
df.to_csv("whole_comment_data_en.csv", encoding = "utf-8-sig", index = False)